In [4]:
import numpy


# row, col: index of the position
# boardSize: size of the board, to determine where the edges are

#returns in bounds neighbors of a given position
def neighborIndex(row, col, boardSize):
    neighbors = numpy.empty(9, dtype=object)
    for i in range(len(neighbors)):
        if (i<3):
            neighbors[i] = (row-1,col-1+i)
        elif (i<6):
            neighbors[i] = (row,col-4+i)
        else:
            neighbors[i] = (row+1,col-7+i)

    invalidIndex = numpy.full(9, -1)

    for i in range(len(invalidIndex)):
        if ( neighbors[i][0] < 0 or neighbors[i][1] < 0 or neighbors[i][0] > boardSize[0]-1 or neighbors[i][1] > boardSize[1]-1):
            invalidIndex[i] = i
    

    invalidIndex = invalidIndex[invalidIndex!=-1]
    invalidIndex = numpy.append(invalidIndex, 4)

    neighbors = numpy.delete(neighbors, invalidIndex)

    return neighbors




# rows: number of rows
# cols: number of cols
# mineCount: number of mines

#creates a consistent minesweeper board
def createBoard(rows,cols,mineCount):  #mineCount < rows*cols + 1

    board = numpy.zeros((rows,cols))

    rng = numpy.random.default_rng()
    minePositions = numpy.arange(rows*cols)
    rng.shuffle(minePositions)
    minePositions = numpy.sort(minePositions[:mineCount])

    for i in range(len(minePositions)):
        m = minePositions[i]
        board[ int((m - m%cols)/cols) ][ m%cols ] = -1

    for i in range(len(board)):
        for j in range(len(board[0])):
            if (board[i][j] == -1):
                neighbors = neighborIndex(i,j,(rows,cols))
                for k in range(len(neighbors)):
                    if (board[ neighbors[k][0] ][ neighbors[k][1] ] != -1):
                        board[ neighbors[k][0] ][ neighbors[k][1] ] += 1
                    

    return board


#print(neighborIndex(0,0, (3,5) ))




newBoard = createBoard(30,16,99)
view = numpy.full((30,16), -2, dtype=float)

print(view)
print("\n")
print(newBoard)




[[-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. 

In [8]:
from queue import Queue

# seed: initial 0 tile 

#returns a list of the positions of all zeros in a connected component
def floodFill(seed, board, view):

    q = Queue(maxsize = 0)
    q.put(seed)
    boardSize = (len(board),len(board[0]))

    while(q.empty() == False):
        n = q.get()
        nX, nY = n

        if (board[nX][nY] == 0 and view[nX][nY] == -2):
            nodeNeighbors = neighborIndex(nX, nY, boardSize)
            for i in range(len(nodeNeighbors)):
                q.put(nodeNeighbors[i])

        view[nX][nY] = board[nX][nY]


    return board, view





# board: the full uncovered board with all mine information
# cover: the board the player sees, with some tiles still covered
# move: indices of a tile a player wishes to click

#returns an updated cover with new tiles revealed, or passes a loss message
def play(board, playerView, move):

    moveRow, moveCol = move
    moveTile = board[moveRow][moveCol]

    if (moveTile == -1):
        playerView[moveRow][moveCol] = moveTile
        return playerView, "GAME OVER"
    
    elif(moveTile == 0):
        '''
        neighbors = neighborIndex(moveRow,moveCol,boardSize)
        neighborValue = numpy.empty(len(neighbors))
        zeroList = []

        for i in range(len(neighborValue)):
            neighborValue[i] = board[neighbors[i][0]][neighbors[i][1]]
            if (neighborValue[i] == 0):
                zeroList.append(neighbors[i]) '''

        board, playerView = floodFill(move, board, playerView)

        return playerView, "CONT" 
    
    else:
        playerView[moveRow][moveCol] = moveTile

        return playerView, "CONT"



updatedView, gameState =  play(newBoard,view,(11,2))
print(updatedView)
print("\n")
print(newBoard)




[[-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2.  3. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. 

In [9]:
def randomAgent(view):

    rng = numpy.random.default_rng()
    nView = numpy.array(view)
    unclickedTiles = numpy.argwhere(nView==-2)
    tileIndex = rng.integers(0,high=len(unclickedTiles)+1)
    move = tuple(unclickedTiles[tileIndex])
    return move

def inputAgent(view):
    move = input()
    move = tuple(int(x) for x in move.split(","))
    return move

In [17]:
# boardSize: the size of the board as a 2-tuple
# mineCount: number of mines

#runs game logic
def  gameRunner(boardSize, mineCount,agent):

    board = createBoard(boardSize[0],boardSize[1],mineCount)
    view = numpy.full((30,16), -2, dtype=float)

    while(True):
        move = agent(view)
        print(move)

        newView, gameState = play(board,view,move)

        view = newView

        print(view)

        if (gameState == "GAME OVER"): 
            print(gameState)
            break
        

    return "success"

gameRunner((30,16),99,randomAgent)

(7, 3)
[[-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2.  1. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -

'success'